In [232]:
import qsharp

In [295]:
%%qsharp


open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Preparation;
open Microsoft.Quantum.Diagnostics;

operation getGraphLength():Int{
    return 4;
}

operation completeCycle(gamma: Double, beta: Double, p: Int): Double {
    return meanCostInverse(QAOA(gamma, beta, p));
}

operation QAOA(gamma:Double, beta:Double, p:Int):Int[]{
    
    //input register
    use register = Qubit[getGraphLength()];
    
    //prepare qubits by creating uniform superposition
    ApplyToEach(H, register);

    //repeat cost and mixer steps p times
    //I think that we might need a new gamma and beta for each p
    for i in 1..p{
        costLayer(register, gamma);
        mixerLayer(register, beta);
    }

    //Read the results as int array
    mutable res = new Int[Length(register)];
    for i in 0..Length(register)-1{
        set res w/= i <- (M(register[i]) == One ? 1 | 0);
    }

    //return the results
    return res;
}

operation mixerLayer(register:Qubit[], beta:Double):Unit{
    ApplyToEach(Rx(2.*beta, _), register); //the '2*beta' is from the qiskit vid https://www.youtube.com/watch?v=YpLzSQPrgSc
}
    
operation costLayer(register:Qubit[], gamma:Double):Unit{
    ApplyToEach(Rz(2.*gamma, _), register);
    for i in 0..Length(register)-1{
         for j in i+1..Length(register)-1{
             Rzz(2.*gamma, register[i], register[j]);
         }
     }
}
   
operation meanCostInverse(register: Int[]) : Double {
    mutable cost = 0.0;
    for i in 0 .. Length(register) - 1 {
        for j in i..Length(register)-1{
            let xi = IntAsDouble(register[i]);
            let xj = IntAsDouble(register[j]);
            set cost -= (1.-xi*xj)/2.; //from https://www.youtube.com/watch?v=aFVnWq3RHYU&t=126s
        }
    }
    return cost;// / IntAsDouble(Length(register));
}

operation meanCostInverse2(register: Int[]) : Double {
    mutable cost = 0.0;
    for i in 0 .. Length(register) - 1 {
        for j in i..Length(register)-1{
            let xi = IntAsDouble(register[i]);
            let xj = IntAsDouble(register[j]);
            if (xi!=xj){
                set cost -= 1.;
            }
        }
    }
    return cost;// / IntAsDouble(Length(register));
}


D:\snippet_.qs(35,19): warning QS3308: Deprecated syntax. Use [] to construct an empty array, or [x, size = n] to construct an array of x repeated n times.


In [296]:
for i in range(10):
    print(QAOA.simulate(gamma=1, beta=1, p=10))

[0, 0, 0, 0]
[0, 0, 0, 0]
[1, 1, 1, 1]
[0, 0, 0, 0]
[0, 0, 0, 0]
[1, 1, 1, 1]
[0, 0, 0, 0]
[0, 1, 1, 1]
[0, 1, 1, 1]
[0, 0, 0, 0]


In [297]:
import numpy as np
from scipy.optimize import minimize

In [298]:
def obj_func(params):
    myres=completeCycle.simulate(gamma=params[0], beta=params[1], p=1);
    print('res= ', myres)
    print('params= ', params)


In [299]:
def getMostFrequent(gamma, beta):
    myDict = {}
    for i in range(1000):
        res = QAOA.simulate(gamma=gamma, beta=beta, p=1)
        res=tuple(res)
        if res in myDict.keys():
            myDict.update({res:myDict.get(res)+1})
        else:
            myDict.update({res:0})
    return np.array(max(myDict))

In [300]:
print(getMostFrequent(11.,0.))

[1 1 1 1]


In [313]:
def compute_expectation(counts):
    """Computes expectation value based on measurement results
    Args:
        counts: (dict) key as bit string, val as count
        graph: networkx graph
    Returns:
        avg: float
             expectation value
    """
    avg = 0
    sum_count = 0
    for bit_string, count in counts.items():
        obj = meanCostInverse2.simulate(register=np.array(bit_string))
        #print(obj, ' =? ', meanCostInverse.simulate(register=np.array(bit_string)))
        avg += obj * count
        sum_count += count
    return avg/sum_count

In [328]:
def getFrequencyDict(gamma, beta):
    myDict = {}
    for i in range(1000):
        res = QAOA.simulate(gamma=gamma, beta=beta, p=1)
        res=tuple(res)
        if res in myDict.keys():
            myDict.update({res:myDict.get(res)+1})
        else:
            myDict.update({res:0})
        #res=np.array(res)
    print(np.array(max(myDict, key=myDict.get)))
    #print(myDict)
    return myDict

In [329]:
print(compute_expectation(getFrequencyDict(1.,1.)))

[0 0 0 0]
-1.2355329949238578


In [330]:
def obj_func4(params):
    return compute_expectation(getFrequencyDict(params[0],params[1]))

In [332]:
res = minimize(obj_func4,[1.0, 1.0], method='COBYLA')
print('all= ', res)

[0 0 0 0]
[0 0 0 0]
[1 1 1 1]
[1 1 1 0]
[1 0 1 1]
[0 0 0 0]
[0 1 0 1]
[0 0 0 0]
[1 1 1 1]
[1 1 1 1]
[1 1 1 1]
[1 1 0 0]
[1 0 0 0]
[0 1 0 1]
[0 0 0 1]
[0 1 1 0]
[0 1 1 0]
[0 0 1 0]
[1 1 0 0]
[0 1 1 0]
[1 1 0 0]
[0 1 0 1]
[0 0 1 1]
[1 1 0 0]
[1 0 0 1]
[0 1 1 0]
[0 0 1 1]
[1 0 0 1]
[0 1 1 0]
[0 1 0 1]
[0 1 1 0]
[0 1 0 1]
[0 0 1 1]
[0 1 1 0]
[0 1 1 0]
all=       fun: -3.529471544715447
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 35
  status: 1
 success: True
       x: array([ 2.90004123, -1.31547053])


In [251]:
def obj_func2 (params):
    myregister=getMostFrequent(params[0], params[1])
    print('params= ', params)
    print('register= ', myregister)
    myRes=meanCostInverse.simulate(register=myregister)
    print('cost= ', myRes)
    return myRes

In [252]:
print(obj_func2([2.,2.]))

params=  [2.0, 2.0]
register=  [1 1 1 1]
cost=  1.0
1.0


In [253]:
res = minimize(obj_func2,[0.0, 0.0], method='COBYLA')
print('all= ', res)

params=  [0. 0.]
register=  [1 1 1 1]
cost=  1.0
params=  [1. 0.]
register=  [1 1 1 1]
cost=  1.0
params=  [0. 1.]
register=  [1 1 1 1]
cost=  1.0
params=  [0.125 0.   ]
register=  [1 1 1 1]
cost=  1.0
params=  [0.    0.125]
register=  [1 1 1 1]
cost=  1.0
params=  [0.015625 0.      ]
register=  [1 1 1 1]
cost=  1.0
params=  [0.       0.015625]
register=  [1 1 1 1]
cost=  1.0
params=  [0.00195312 0.        ]
register=  [1 1 1 1]
cost=  1.0
params=  [0.         0.00195312]
register=  [1 1 1 1]
cost=  1.0
params=  [0.00024414 0.        ]
register=  [1 1 1 1]
cost=  1.0
params=  [0.         0.00024414]
register=  [1 1 1 1]
cost=  1.0
params=  [5.e-05 0.e+00]
register=  [1 1 1 1]
cost=  1.0
params=  [0.e+00 5.e-05]
register=  [1 1 1 1]
cost=  1.0
all=       fun: 1.0
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 13
  status: 1
 success: True
       x: array([0., 0.])


In [254]:
def obj_func3 (params):
    myregister=QAOA.simulate(gamma=params[0], beta=params[1], p=1)
    #print('params= ', params)
    #print('register= ', myregister)
    myRes=meanCostInverse.simulate(register=myregister)
    print('cost= ', myRes)
    return myRes

In [255]:
print(obj_func3([2.,2.]))

cost=  1.0
1.0


In [256]:
for i in range(5):
    res = minimize(obj_func3,[i, i+3.0], method='COBYLA')
    print('i= ',i,' i+3= ', i+3)
    print('all= ', res)

cost=  -0.25
cost=  0.875
cost=  -0.25
cost=  -0.25
cost=  -0.25
cost=  -0.25
cost=  1.0
cost=  -0.25
cost=  -0.25
cost=  0.125
cost=  -0.25
cost=  -0.25
cost=  1.0
cost=  0.5
cost=  0.5
cost=  -0.25
cost=  -0.25
cost=  -0.25
cost=  -0.25
i=  0  i+3=  3
all=       fun: -0.25
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 19
  status: 1
 success: True
       x: array([0., 3.])
cost=  1.0
cost=  0.5
cost=  1.0
cost=  -0.25
cost=  1.0
cost=  -0.25
cost=  -0.25
cost=  0.875
cost=  0.125
cost=  -0.25
cost=  -0.25
cost=  0.0
cost=  -0.25
cost=  0.625
cost=  -0.25
cost=  -0.25
cost=  0.875
cost=  0.125
cost=  -0.25
cost=  0.875
cost=  0.625
cost=  -0.25
cost=  -0.25
cost=  0.5
cost=  0.625
cost=  0.375
cost=  -0.25
i=  1  i+3=  4
all=       fun: -0.25
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 27
  status: 1
 success: True
       x: array([2.70719847, 3.29285332])
cost=  -0.25
cost=  -0.25
cost=  -0.25
cost=  0.125
cost=  -0.25
cost=  